# Generar Matriz de Distancias

### Instrucciones:

1.- Primero cargar en la carpeta de la izquierda los inputs del programa que son los siguientes
> a.- Origenes: Es una capa shapefile de poligonos con los campos que quiero calcular.  
Tiene que tener los nombres de los campos en una columna llamada "idest".  
En general esta capa es una parte del espcul.  
Nuna cargar mas de una zona porque la API gratuita no lo permite.   

> b.- Destinos: Es una capa shapefile de puntos con los destinos.  
Debe tener la columna "Localidad" con las localidades de los puertos, molinos o acopios de la capa.  
  
2.- Una vez cargadas las dos capas, ir al menu "Entorno de ejecución" y seleccionar "Reiniciar y ejecutar todo".  

3.- Una vez hecho esto dejar procesar la info y cuando termine de correr el script, aparecerán tildes en todas las casillas y abajo y aparecerá un archivo excel con el nombre "matrix.xlsx" en la misma carpeta de la izquierda donde cargamos los inputs. Hacer click en los 3 puntitos y descargar el archivo con los resultados.


In [55]:
import pandas as pd
import geopandas as gpd
import osmnx as ox
import datetime as datetime
import os
from pathlib import Path
import time
import requests

In [ ]:
try:
  espcul_gdf = gpd.read_file('./inputs/origenes.shp')

except Exception as e:
  espcul_gdf = gpd.read_file('./inputs/origenes.gpkg')

espcul_gdf = espcul_gdf.loc[espcul_gdf.campania == '24/25']

espcul_gdf = espcul_gdf[['zona', 'idest', 'geometry']]

print(espcul_gdf.shape)
espcul_gdf.head()

In [ ]:
zonas_list = espcul_gdf.zona.unique()
zonas_list

In [58]:
# espcul_gdf = espcul_gdf.loc[espcul_gdf.zona.isin(['LA PAMPA'])]

In [ ]:
espcul_gdf = gpd.GeoDataFrame(espcul_gdf, geometry='geometry').reset_index()
espcul_gdf['geometry'] = espcul_gdf.buffer(0.0001)
espcul_dissolve = espcul_gdf.dissolve(by='idest').reset_index()

print(espcul_dissolve.shape)
espcul_dissolve.head()

In [ ]:
espcul_dissolve['x']= espcul_dissolve['geometry'].centroid.x
espcul_dissolve['y']= espcul_dissolve['geometry'].centroid.y
espcul_dissolve['centres']= espcul_dissolve['geometry'].centroid
espcul_centroids = espcul_dissolve.loc[:,['idest','x','y','centres']].copy()
espcul_centroids = espcul_centroids.rename(columns = {'centres': 'geometry'})
espcul_centroids = espcul_centroids.to_crs('EPSG:4326')
espcul_centroids.head()

In [61]:
espcul_centroids.to_file(driver = "GPKG",filename='./inputs/centoids.gpkg', encoding='utf-8', index=False)

In [62]:
espcul_coord = {}

for i in range(0, len(espcul_centroids)):
    origin = espcul_centroids['idest'][i]
    x = espcul_centroids['geometry'][i].x
    y = espcul_centroids['geometry'][i].y
    coords = (y, x)
    espcul_coord[origin]=coords
# espcul_coord

In [ ]:
# Opcion para hacer la matriz con todos los destinos
try:
  destinos_gdf = gpd.read_file('./inputs/destinos.shp')

except Exception as e:
  destinos_gdf = gpd.read_file('./inputs/destinos.gpkg')


print(destinos_gdf.shape)
destinos_gdf.head()

destinos_coord = {}

for i in range(len(destinos_gdf)):
    destino = destinos_gdf['Localidad'][i]
    punto = destinos_gdf['geometry'][i].centroid  # Calcula el centroide
    coords = (punto.y, punto.x)
    destinos_coord[destino] = coords

list(destinos_coord.items())[:4]

The centroids may be away form actual street network. Use osmnx to find the closest node on OSM network for routing

In [64]:
arr = espcul_gdf.to_crs('EPSG:4326').total_bounds
tupla = tuple(arr)
# print(tupla)

In [ ]:
G = ox.graph.graph_from_bbox(tupla, network_type='drive')
nodes = G.nodes()

In [ ]:
espcul_coord_snapped = {}
for name, coords in espcul_coord.items():
    node = ox.distance.nearest_nodes(G, coords[1], coords[0])
    info = nodes[node]
    espcul_coord_snapped[name] = (info['y'], info['x'])

espcul_coord_snapped

In [67]:
def get_driving_distance(source_coordinates, dest_coordinates, port=5001):
    """
    Returns the driving distance in kilometers between two coordinates using a local OSRM server.

    Parameters:
    - source_coordinates: tuple (lat, lon)
    - dest_coordinates: tuple (lat, lon)
    - port: local port where OSRM server is running (default is 5000)

    Returns:
    - distance in kilometers (float)
    - returns -9999 if request fails
    """
    base_url = f"http://localhost:{port}/route/v1/driving"
    # OSRM expects lon,lat
    src_lon, src_lat = source_coordinates[1], source_coordinates[0]
    dst_lon, dst_lat = dest_coordinates[1], dest_coordinates[0]

    url = f"{base_url}/{src_lon},{src_lat};{dst_lon},{dst_lat}?overview=false"

    try:
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            distance = data['routes'][0]['distance']  # in meters
            return distance / 1000  # return in kilometers
        else:
            print(f"Request failed. Status code: {response.status_code}")
            return -9999
    except Exception as e:
        print(f"Error connecting to OSRM server: {e}")
        return -9999

In [68]:
rows = []

for k1, v1 in espcul_coord_snapped.items():
    for k2, v2 in destinos_coord.items():
        origin = v1
        destino = v2
        distance = get_driving_distance(origin, destino, port=5001)
        
        rows.append({
            'campo': k1,
            'campo_x': v1[1],
            'campo_y': v1[0],
            'destino': k2,
            'destino_x': v2[1],
            'destino_y': v2[0],
            'distancia': int(distance)
        })

df = pd.DataFrame(rows)

# df.head()

In [ ]:
resultados = pd.merge(df, espcul_dissolve[['idest', 'zona']], left_on='campo', right_on='idest', how='left').drop(columns='idest')
resultados = resultados[['zona', 'campo', 'campo_x', 'campo_y', 'destino', 'destino_x', 'destino_y', 'distancia']]
resultados.head()

In [ ]:
print(f'df original: {df.shape}')
print(f'df zonas: {resultados.shape}')

In [70]:
path = "./outputs"

if not os.path.exists(path):
    os.makedirs(path)

# df.to_csv('matrix.csv', index=False)
resultados.to_excel('./outputs/matriz_de_distancias.xlsx', index=False)